# C-mean Clustering
Fuzzy c-Means Clustering} (FMC) is a clustering method which, unlike Hard k-Means Clustering, allows a data sample to more or less belong to one or more clusters. In this notebook, we will try to develop a semi-supervised C-mean Clustering.

## Loading the packages

In [1]:
import numpy as np
import matplotlib.pyplot as plt

import random
import operator
import math
colors = ['b', 'orange', 'g', 'r', 'c', 'm', 'y', 'k', 'Brown', 'ForestGreen']

PATH_TO_DATASET = '' # Not Given Yet 

%matplotlib inline

## The dataset
The following script allows you to create a 2D dataset by using the mouse. The fift first click adds points belonging to class A (blue), the 5th following click adds points belonging to class B (red) and the next click add unclassified points. You can create as many points as you desire. The final dataset will contain hence three values per point: x coordinate (-1 ≤ x ≤ 1), y coordinate (-1 ≤ y ≤ 1) and the class ∈ {0,1,2} where 0 represent unclassified points.

In [2]:
%matplotlib notebook

fig = plt.figure(figsize=(6,6))
plt.title("Input Dataset")
plt.xlim((-1.2,1.2))
plt.ylim((-1.2,1.2))

dataset = []

i = 0
def onclick(event):
    global dataset
    global i
    cx = event.xdata
    cy = event.ydata
    classe = 1 if(i<10) else 2 if(i<20) else 0
    i=i+1
    dataset.append((cx, cy, classe))

    plt.scatter(cx, cy, c=['k', 'b', 'r'][classe], s=100, lw=0)
    plt.grid(True)

cid = fig.canvas.mpl_connect('button_press_event', onclick)


<IPython.core.display.Javascript object>

In [3]:
import numpy as np

def random_positive_semidefinite_matrix(d):
    Q = np.random.randn(d, d)
    eigvals = np.random.rand(d)
    return Q.T @ np.diag(eigvals) @ Q
    
    while True:
        A = np.random.rand(d, d)
        A += A.T
        if np.all(np.linalg.eigvals(A) > 0):
            return A

def make_spiral_clusters(c, cluster_size, n_noise, d=2):
    angle = np.linspace(0, 2*np.pi, c, endpoint=False)
    radius = np.linspace(10, 30, c)
    vx = np.cos(angle)*radius
    vy = np.sin(angle)*radius
    v = np.stack([vx, vy], axis=1)

    covariances = np.array([random_positive_semidefinite_matrix(d) for _ in range(c)])

    x = np.concatenate([np.random.multivariate_normal(v[i], covariances[i], cluster_size)
        for i in range(c)], axis=0)

    u = np.random.rand(n_noise, d)
    noise = np.min(x, axis=0)*u + (1 - u)*np.max(x, axis=0)

    x = np.concatenate([x, noise], axis=0)

    return x, v


dataset, true_centers = make_spiral_clusters(5, 50, 15)

labels = [1]*int(len(dataset)/5) + [2]*int(len(dataset)/5) + [1]*int(len(dataset)/5) + [2]*int(len(dataset)/5)+ [1]*int(len(dataset)/5)
for i in range(50):
    labels[np.random.randint(len(labels))] = 0

fig2 = plt.figure(figsize=(6,6))
plt.title("Output Dataset")

plt.scatter(dataset[:,0], dataset[:,1], c=[(['b', 'g', 'r', 'c', 'm', 'y', 'k', 'w'])[int(label)] for label in labels])
plt.grid()
plt.show()
print(dataset)

<IPython.core.display.Javascript object>

[[ 9.58150966e+00  1.74982866e+00]
 [ 1.01931423e+01 -1.08302638e+00]
 [ 9.43984917e+00 -1.93923140e+00]
 [ 1.06764358e+01 -4.67365154e-01]
 [ 9.99739661e+00 -3.31858984e-01]
 [ 9.94608862e+00 -7.80496430e-01]
 [ 1.03552508e+01  4.11852519e-02]
 [ 9.78939391e+00  1.29840487e+00]
 [ 1.04107393e+01  5.40179800e-01]
 [ 1.06416964e+01  2.12252763e-01]
 [ 1.01579646e+01  3.12039596e-01]
 [ 9.71762966e+00 -6.92328431e-01]
 [ 1.05091861e+01 -9.41852475e-01]
 [ 9.94918792e+00 -5.84594232e-01]
 [ 9.97684139e+00  3.21792362e-01]
 [ 9.48966357e+00 -7.77592827e-03]
 [ 9.93197051e+00 -7.39635982e-01]
 [ 1.02580608e+01  9.13094286e-01]
 [ 9.81488919e+00 -5.59041957e-01]
 [ 1.08728111e+01 -5.54743845e-01]
 [ 9.98078340e+00 -6.98104976e-01]
 [ 1.03885656e+01  8.78748487e-02]
 [ 1.05406516e+01 -1.45721618e+00]
 [ 9.31377462e+00  1.95122100e-02]
 [ 1.09108492e+01 -3.45345954e-01]
 [ 9.29744446e+00  1.71618163e+00]
 [ 1.13407557e+01  2.82508299e-02]
 [ 9.45543776e+00 -1.25095554e-01]
 [ 1.00239300e+01  1

In [4]:
dataset = np.array(dataset)
labels = np.expand_dims(labels, axis=1)
dataset = np.append(dataset, labels, axis=1)
fig2 = plt.figure(figsize=(6,6))
plt.title("Output Dataset")

plt.scatter(dataset[:,0], dataset[:,1], c=[(['b', 'g', 'r', 'c', 'm', 'y', 'k', 'w'])[int(label)] for label in labels])
plt.grid()
plt.show()
print(dataset)

<IPython.core.display.Javascript object>

[[ 9.58150966e+00  1.74982866e+00  1.00000000e+00]
 [ 1.01931423e+01 -1.08302638e+00  1.00000000e+00]
 [ 9.43984917e+00 -1.93923140e+00  1.00000000e+00]
 [ 1.06764358e+01 -4.67365154e-01  1.00000000e+00]
 [ 9.99739661e+00 -3.31858984e-01  1.00000000e+00]
 [ 9.94608862e+00 -7.80496430e-01  1.00000000e+00]
 [ 1.03552508e+01  4.11852519e-02  1.00000000e+00]
 [ 9.78939391e+00  1.29840487e+00  1.00000000e+00]
 [ 1.04107393e+01  5.40179800e-01  0.00000000e+00]
 [ 1.06416964e+01  2.12252763e-01  1.00000000e+00]
 [ 1.01579646e+01  3.12039596e-01  1.00000000e+00]
 [ 9.71762966e+00 -6.92328431e-01  1.00000000e+00]
 [ 1.05091861e+01 -9.41852475e-01  1.00000000e+00]
 [ 9.94918792e+00 -5.84594232e-01  1.00000000e+00]
 [ 9.97684139e+00  3.21792362e-01  1.00000000e+00]
 [ 9.48966357e+00 -7.77592827e-03  1.00000000e+00]
 [ 9.93197051e+00 -7.39635982e-01  1.00000000e+00]
 [ 1.02580608e+01  9.13094286e-01  1.00000000e+00]
 [ 9.81488919e+00 -5.59041957e-01  1.00000000e+00]
 [ 1.08728111e+01 -5.54743845e-

## Membership Matrix
The Membership matrix lists all the data samples membership grades to a particular cluster. Here we first randomly initialize it.

In [5]:
def initializeMembershipMatrix(dataset, k):
    membership_mat = list()
    for i in range(len(dataset)):
        random_num_list = [random.random() for i in range(k)]
        summation = sum(random_num_list)
        temp_list = [x/summation for x in random_num_list]
        membership_mat.append(temp_list)
    return membership_mat

## Cluster Center
Here we calculate each cluster center depending on the membership grades.

In [6]:
def calculateClusterCenter(dataset, membership_mat, k, fuzzy_param):
    cluster_mem_val = list(zip(*membership_mat))
    cluster_centers = list()
    for j in range(k):
        x = cluster_mem_val[j]
        xraised = [e ** fuzzy_param for e in x]
        denominator = sum(xraised)
        temp_num = list()
        for i in range(len(dataset)):
            data_point = dataset[i]
            prod = [xraised[i] * val for val in data_point]
            temp_num.append(prod)
        numerator = map(sum, zip(*temp_num))
        center = [z/denominator for z in numerator]
        cluster_centers.append(center)
    return cluster_centers

## Semi-supervised C-Mean

In [7]:
def evalConvergence(mb, previous_mb, threshold):
    return (np.absolute(np.array(mb) - np.array(previous_mb))).max() < threshold
        

In [8]:
def initializeMembershipMatrixSupervised(dataset, k):
    membership_mat_supervised = list()
    for i in range(len(dataset)):
        if (dataset[i][2] == 0):
            random_num_list = [0 for i in range(k)]
        else:
            random_num_list = [1 if((i+1)==dataset[i][2]) else 0 for i in range(k)]
        
        membership_mat_supervised.append(random_num_list)
    return membership_mat_supervised

In [9]:
def evalAccuracy(dataset, k):
    pass

In [10]:
def calculateClusterCenterSup(dataset, membership_mat, membership_mat_sup,  k, fuzzy_param):
    cluster_centers = []
    for i in range(k):
        for n in range(len(dataset)):
            denum = abs(membership_mat[n][i] - membership_mat_sup[n][i]) ** fuzzy_param
            num = denum*dataset[n]#dataset[n, :-1] ? 
        cluster_centers.append(num/denum)
    return cluster_centers

In [11]:
def updateMembershipValueSup(dataset, membership_mat, membership_mat_sup, cluster_centers, fuzzy_param, k):
    def d(n, c):
        return np.linalg.norm(list(map(operator.sub, dataset[n], cluster_centers[c])))** 2
    p = 1/(fuzzy_param- 1)
    
    for i in range(k):
        for n in range(len(dataset)):  
            membership_mat[n][i] = membership_mat_sup[n][i] + (1 - np.sum(membership_mat_sup[n])) * d(n, i) ** p / np.sum([d(n,j) for j in range(k)])
    return membership_mat

In [12]:
import numpy as np
import scipy.spatial
import copy

def pairwise_squared_distances(A, B):
    return scipy.spatial.distance.cdist(A, B)**2

def calculate_covariances(x, u, v, m):
    c, n = np.array(u).shape
    d = np.array(v).shape[1]
    
    um = np.array(u)**m

    covariances = np.zeros((c, d, d))

    for i in range(c):
        xv = x - v[i]
        uxv = um[i, :, np.newaxis]*xv
        covariances[i] = np.einsum('ni,nj->ij', uxv, xv)/np.sum(um[i])
    
    return covariances

# Partition Coefficient
def pc(x, u, v, m):
    c, n = np.array(u).shape
    return np.square(np.array(u)).sum()/n

# Fuzzy Hyperbolic Volume
def fhv(x, u, v, m):
    covariances = calculate_covariances(x, u, v, m)
    return sum(np.sqrt(np.linalg.det(cov)) for cov in covariances)

# Xie-Beni Index
def xb(x, u, v, m):
    n = np.array(x).shape[0]
    c = np.array(v).shape[0]

    um = np.array(u)**m
    
    d2 = pairwise_squared_distances(x, v)
    v2 = pairwise_squared_distances(v, v)
    
    v2[v2 == 0.0] = np.inf

    return np.sum(um.T*d2)/(n*np.min(v2))

def checkKnownEntriesBetter(dataset, labels, c):
    res = np.zeros((c, 2))
    for cluster in range(c) :
        for i in range(len(dataset)):
            if(dataset[i][-1] != 0 and labels[i] == cluster):
                res[cluster][int(dataset[i][-1] - 1)] += 1
        
   # print("res1", res)
    
    for cluster in range(c):
        c_total = res[cluster][0] + res[cluster][1]
        res[cluster][0] /= c_total
        res[cluster][1] /= c_total

  #  print("res2", res)
    return res
        
    

def checkKnownEntries(x, labels, c):
    
    # 4 columns for each cluster :
    # |# class0 |# class1 | %0 | %1 |
    min_c = min(labels)
    res = [[0 for i in range(4)] for j in range(c)]
    for i in range(len(x)):
        # If the class is known (i.e. not 0)
        if(x[i][-1]!= 0):
            print(str(x[i][-1]))
            res[labels[i] - min_c][int(x[i][-1])-1] = res[labels[i] - min_c][int(x[i][-1])] + 1
            
    for i in range(c):
        if((res[i][0] + res[i][1]) != 0 and (res[i][0] + res[i][1]) != 0):
            res[i][2] = res[i][0] / (res[i][0] + res[i][1])        
            res[i][3] = res[i][1] / (res[i][0] + res[i][1])
        else :
            res[i][2] = 0.5
            res[i][3] = 0.5
    #print(res)
        
    return res


In [13]:
import numpy as np
from cvi import *

def fcm_get_u(x, v, m):
    distances = pairwise_squared_distances(x, v)
    nonzero_distances = np.fmax(distances, np.finfo(np.float64).eps)
    inv_distances = np.reciprocal(nonzero_distances)**(1/(m - 1))
    return inv_distances.T/np.sum(inv_distances, axis=1)

def fcm(x, c, m=2.0, v=None, max_iter=100, error=0.05):
    if v is None: v = x[np.random.randint(x.shape[0], size=c)]
    u = fcm_get_u(x, v, m)
    for iteration in range(max_iter):
        u_old = u
        um = u**m
        v = np.dot(um, x)/np.sum(um, axis=1, keepdims=True)
        u = fcm_get_u(x, v, m)
        if np.linalg.norm(u - u_old) < error: break
    return u, v

In [14]:
def getClass(dataset, labels):
    count_class = np.zeros((len(np.unique(labels)), 2))
    newLabels = copy.copy(labels)
    for i in range(len(dataset)):
        if(dataset[i][-1] != 0):
            count_class[labels[i]][int(dataset[i][-1] - 1)] += 1
    
    print(count_class)
    
    for c in range(len(count_class)):     
        if count_class[c][0] < count_class[c][1]:
            cl = 1 
        elif count_class[c][0] > count_class[c][1]:
            cl = 2 
        else :
            cl = 0
            
        for i in range(len(labels)):
            if labels[i] == c:
                newLabels[i] = cl 
            
    return newLabels

def extractCluster(c, dataset, labels):
    print('Good cluster :', c)
 #   print('dataset', dataset)
    cluster = np.empty((0, len(dataset[0])), float)
    indices = []
    for i in range(len(dataset)):
        if(labels[i] == c):
            cluster = np.vstack((cluster, dataset[i]))
            indices.append(i)
#    print('Cluster', cluster)
    return cluster, indices
    

In [80]:
def getAccuracy(true_labels, labels):
    same = 0
    for t, p in zip(true_labels, labels):
        if t == p:
            same += 1
    return same / len(true_labels) * 100

## Main Execution 
blablabla

In [68]:
def updateMembershipValue(dataset, membership_mat, cluster_centers, fuzzy_param, k):
    p = float(2/(fuzzy_param-1))
    for i in range(len(dataset)):
        x = dataset[i]
        distances = [np.linalg.norm(list(map(operator.sub, x, cluster_centers[j]))) for j in range(k)]
        for j in range(k):
            den = sum([math.pow(float(distances[j]/distances[c]), p) for c in range(k)])
            membership_mat[i][j] = float(1/den)       
    return membership_mat


def getClusters(dataset, membership_mat):
    cluster_labels = list()
    for i in range(len(dataset)):
        max_val, idx = max((val, idx) for (idx, val) in enumerate(membership_mat[i]))
        cluster_labels.append(idx)
    return cluster_labels


def fuzzyCMeansClustering(dataset, k = 2, threshold = 0.1, fuzzy_param = 2.00, maxSteps = 100000):
    # Membership Matrix
    membership_mat = initializeMembershipMatrix(dataset, k)
    previous_mb = None
    steps = 0
    
    while previous_mb is None or not evalConvergence(membership_mat, previous_mb, threshold) or steps >= maxSteps:
        cluster_centers = calculateClusterCenter(dataset[:,:-1], membership_mat, k, fuzzy_param)
        membership_mat = updateMembershipValue(dataset[:,:-1], membership_mat, cluster_centers, fuzzy_param, k)
        cluster_labels = getClusters(dataset[:,:-1], membership_mat)
        previous_mb = membership_mat
        steps += 1
    return cluster_labels, cluster_centers, membership_mat

def clusterize(dataset):
    result_labels = np.empty((len(dataset), len(dataset[0]) + 1), float)
    result_mb     = np.empty((0, len(dataset[0])), float)
    result_ctrs   = np.empty((0, len(dataset[0])), float)
    temp_data     = copy.copy(dataset)
    found_clusters = 0
    result_index = 0
    c = 2
    done = False
    while(not done and c < math.sqrt(len(dataset))): 
        #labels, centers, mb = fuzzyCMeansClustering(temp_data, k = c)
        mb, centers = fcm(temp_data[:,:-1], c)
        mb_t = mb.T

        labels = getClusters(temp_data[:,:-1], mb_t)
        # First, check supervised values
        # TOFIX : Bordel with cluster indexes, won't work.
        sup_verif = checkKnownEntriesBetter(temp_data, labels, c)
     #   print("verif: ", sup_verif)
        cluster_ok = []
        for i in range(c):
            if(sup_verif[i][0] < 0.1 or sup_verif[i][0] > 0.90):
                cluster_ok.append(i)
        print('cluster ok ', cluster_ok)
        print(sup_verif)
        print("Len remaining data", len(temp_data))

        if(len(cluster_ok) == 0):
            # If no cluster is good enough (with known values) restart with more clusters
            c =c + 1
        
        else:
            # Check our values aren't just luckily in same clusters using unsup CVI
            fhv_s = fhv(x = temp_data[:,:-1], v = centers, u = mb, m =2)
            pc_s  = pc(temp_data[:,:-1], mb, centers, 2)
            xb_s  = xb(x = temp_data[:,:-1], u = mb, v = centers, m = 2)
            print('fhvs_', fhv_s)
            print('pc_s', pc_s)
            print('xb_s', xb_s)
            # If clustering is very good, stop
            if(pc_s > 0.9 and xb_s < 0.1 and fhv_s < 10):#fhv_s > 0.1 or 
                done = 1
                break
            else:
                for c in cluster_ok :
                    cluster, indices = extractCluster(c, temp_data, labels)
                    print(len(result_labels))
                    print(len(cluster))
                    for i in range(len(cluster)):
                        result_labels[result_index] = np.append(cluster[i], found_clusters)
                        result_index += 1
                    for i in range(result_index, len(result_labels)):
                        result_labels[i] = [0, 0, 0, 0]
                 #   result_labels = np.append(result_labels[found_clusters], cluster)
                 #   result_labels = np.reshape(result_labels[found_clusters],(len(result_labels)/3, 3))
                    temp_data = np.delete(temp_data, indices, 0)
                    print(result_labels)
                    found_clusters += 1
            c = c - len(cluster_ok) + 1
            continue
            
    return labels, centers, mb

labels, centers, mb = clusterize(dataset)
labels = getClass(dataset, labels)
#labels, centers, mb = fuzzyCMeansClustering(dataset, k=2)

#mb, centers = fcm(dataset[:,:-1], 4)
#print(len(mb))
#print(len(mb[0]))

#labels = getClusters(dataset[:,:-1], mb.T )
#print(len(labels))
#print(len(dataset))
#print(mb)

#centers = np.array(centers)
# Visualize the test data

fig2 = plt.figure(figsize=(6,6))
plt.title("Output Dataset")
plt.scatter(dataset[:,0], dataset[:,1], c=[(['b', 'g', 'r', 'c', 'm', 'y', 'k', 'w'])[int(label)] for label in labels])
plt.scatter(centers[:,0], centers[:,1], c='y')
plt.grid()
plt.show()

cluster ok  []
[[0.65648855 0.34351145]
 [0.52272727 0.47727273]]
Len remaining data 265
cluster ok  []
[[0.27272727 0.72727273]
 [0.65116279 0.34883721]
 [0.7826087  0.2173913 ]]
Len remaining data 265
cluster ok  []
[[0.84615385 0.15384615]
 [0.28888889 0.71111111]
 [0.77777778 0.22222222]
 [0.56666667 0.43333333]]
Len remaining data 265
cluster ok  [1, 2]
[[0.28888889 0.71111111]
 [1.         0.        ]
 [0.09302326 0.90697674]
 [0.84210526 0.15789474]
 [0.77777778 0.22222222]]
Len remaining data 265
fhvs_ 11.989178545309086
pc_s 0.9450658522841471
xb_s 0.020081710114473197
Good cluster : 1
265
55
[[ 9.58150966  1.74982866  1.          0.        ]
 [10.19314227 -1.08302638  1.          0.        ]
 [ 9.43984917 -1.9392314   1.          0.        ]
 ...
 [ 0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.        ]]
Good cluster : 2
265
50
[[ 9.58150966  1.74982866  1.          0.        ]
 [10

ValueError: max() arg is an empty sequence

## Results and Analyse

In [ ]:
from sklearn.metrics import confusion_matrix
import itertools
def plot_confusion_matrix(cm, title='Confusion matrix', cmap=plt.cm.Blues):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    labels_names = ["Woman", "Man"]
    tick_marks = np.arange(len(labels_names))
    plt.xticks(tick_marks, labels_names, rotation=45)
    plt.yticks(tick_marks, labels_names)
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        pl.text(j, i, format(cm[i, j]),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")
plot_confusion_matrix(conf_mat)

In [ ]:
precision = (conf_mat[0][0]/(conf_mat[0][0] + conf_mat[1][0]) + conf_mat[1][1]/(conf_mat[1][1] + conf_mat[0][1]))/2
print("Precision: ", precision)
recall = (conf_mat[0][0]/(conf_mat[0][0] + conf_mat[0][1]) + conf_mat[1][1]/(conf_mat[1][1] + conf_mat[1][0]))/2
print("Recall: ", recall)
f1score = 2*(precision*recall)/(precision+recall)
print("F1-Score: ", f1score)